# Graph Neural Networks (GNNs)

## Quelle der Daten

https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database?resource=download (zuletzt aufgerufen 01/2024)
    
M.E.H. Chowdhury, T. Rahman, A. Khandakar, R. Mazhar, M.A. Kadir, Z.B. Mahbub, K.R. Islam, M.S. Khan, A. Iqbal, N. Al-Emadi, M.B.I. Reaz, M. T. Islam, “Can AI help in screening Viral and COVID-19 pneumonia?” IEEE Access, Vol. 8, 2020, pp. 132665 - 132676.

Rahman, T., Khandakar, A., Qiblawey, Y., Tahir, A., Kiranyaz, S., Kashem, S.B.A., Islam, M.T., Maadeed, S.A., Zughaier, S.M., Khan, M.S. and Chowdhury, M.E., 2020. Exploring the Effect of Image Enhancement Techniques on COVID-19 Detection using Chest X-ray Images.

## Installation der Bibliotheken

In [ ]:
# Bibliotheken importieren, die für das Laden, Vorverarbeiten und Modellieren der Daten benötigt werden.
import os

import cv2

import matplotlib.pyplot as plt

import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras import Model, layers

from tensorflow.keras.layers import Dense, Input

import keras

import requests

import tempfile

import shutil

## Download der Daten

In [ ]:
# URL des Datensatzes festlegen und temporäres Verzeichnis für den Download erstellen.
# load dataset from online resource, given here: https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database
ds_url = "https://drive.usercontent.google.com/download?id=1bum9Sehb3AzUMHLhBMuowPKyr_PCrB3a&export=download&confirm=1"
temp_dir = tempfile.mkdtemp()
print(f"Temporary file-directory for saving dataset: '{temp_dir}'\n")

In [ ]:
# for downloading from google-drive, use method described
# elsewhere (https://stackoverflow.com/a/39225272)
# with some modifications

def download_file_from_google_drive(URL, destination_dir):
    # Funktion zum Herunterladen einer Datei von Google Drive
    session = requests.Session()

    response = session.get(URL, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {"confirm": token}
        response = session.get(URL, params=params, stream=True)

    save_response_content(response, destination_dir)


def get_confirm_token(response):
    # Funktion zum Abrufen des Bestätigungstokens für den Download
    for key, value in response.cookies.items():
        if key.startswith("download_warning"):
            return value

    return None


def save_response_content(response, destination_dir):
    # Funktion zum Speichern des Dateiinhaltes
    CHUNK_SIZE = 32768

    with open(destination_dir + "/data.zip", "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)

# Die Datei von Google Drive herunterladen
download_file_from_google_drive(ds_url, temp_dir)

print("Folder structure inside 'tempdir':\n")
print(os.listdir(temp_dir))

In [ ]:
# Entpacken der heruntergeladenen Zip-Datei
shutil.unpack_archive(filename=temp_dir + "/data.zip", extract_dir=temp_dir)
print("Folder structure inside 'tempdir':\n")
print(os.listdir(temp_dir))

## Einlesen und Präprozessierung der Daten

In [ ]:
# Pfade zu den Bilderverzeichnissen festlegen und Anzahl der Bilder ausgeben
main_path = temp_dir + "/COVID-19_Radiography_Dataset"
print(os.listdir(main_path))

covid_dir = os.path.join(main_path, "COVID/images")
normal_dir = os.path.join(main_path, "Normal/images")

print("Anzahl Bilder mit COVID:", len(os.listdir(covid_dir)))
print("Anzahl normaler Bilder:", len(os.listdir(normal_dir)))

In [ ]:
# Beispielbild laden und anzeigen
example_image = cv2.imread(os.path.join(covid_dir, "COVID-1.png"))

plt.imshow(example_image)

In [ ]:
# Form des Beispielbildes ausgeben
print(example_image.shape)

In [ ]:
# Funktion zum Laden und Vorverarbeiten von Bildern aus einem Verzeichnis
def loadImages(dir, size, label):
  images = []
  labels = []

  for i in range(len(size)):
    img_path = dir + "/" + size[i]
    img = cv2.imread(img_path)
    img = img / 255.0
    img = cv2.resize(img, (100, 100))
    images.append(img)
    labels.append(label)

  images = np.asarray(images)

  return images, labels

In [ ]:
# Laden und Beschriften von COVID-Bildern (10% des Datensatzes)
# (kann bei Bedarf / je nach Verfügbarkeit der Ressourcen angepasst werden)
covid_images, covid_labels = loadImages(covid_dir, os.listdir(covid_dir)[:int(0.1*len(os.listdir(covid_dir)))], 1)
print("Covid cases:\n")
print(len(covid_images), len(covid_labels))

In [ ]:
# Ausgabe der ersten 10 Covid-Labels
print(covid_labels[0:10])

In [ ]:
# Laden und Beschriften von normalen Bildern (10% des Datensatzes)
# (kann bei Bedarf / je nach Verfügbarkeit der Ressourcen angepasst werden)
normal_images, normal_labels = loadImages(normal_dir, os.listdir(normal_dir)[:int(0.1*len(os.listdir(normal_dir)))], 0)
print("Normal cases:\n")
print(len(normal_images), len(normal_labels))

In [ ]:
# Ausgabe der ersten 10 Labels von normalen Fällen
print(normal_labels[0:10])

## Aufteilung der Daten

In [ ]:
# Kombinieren der COVID- und normalen Bilddaten und Labels
x = np.r_[covid_images, normal_images]

y = np.r_[covid_labels, normal_labels]

In [ ]:
# Aufteilen der Daten in Trainings- und Testsets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15)

## Umwandlung der Bilder in Graphen

In [ ]:
def image_to_graph(image):
  # Die Form des Bildes abrufen (Höhe, Breite, Kanäle)
  h, w, c = image.shape

  # Indizes für jeden Pixel erstellen und in eine 2D-Struktur umwandeln
  node_indices = np.arange(h * w).reshape((h, w))

  # Kanten für benachbarte Pixel in horizontaler Richtung (rechts und links) erstellen
  edges_r = np.stack((node_indices[:, :-1].reshape((-1)), node_indices[:, 1:].reshape((-1))), axis = 0)
  edges_l = np.stack((node_indices[:, 1:].reshape((-1)), node_indices[:, :-1].reshape((-1))), axis = 0)
  # Kanten für benachbarte Pixel in vertikaler Richtung (unten und oben) erstellen
  edges_d = np.stack((node_indices[:-1, :].reshape((-1)), node_indices[1:, :].reshape((-1))), axis = 0)
  edges_u = np.stack((node_indices[1:, :].reshape((-1)), node_indices[:-1, :].reshape((-1))), axis = 0)

  # Alle Kanten zusammenführen
  edges = np.concatenate((edges_r, edges_l, edges_d, edges_u), axis = 1).astype(np.int32)
  # Kanten in einen TensorFlow Tensor umwandeln
  edges = tf.convert_to_tensor(edges)

  # Gewichte für jede Kante erstellen (hier: alle Gewichte sind 1)
  edge_weights = tf.ones(shape = edges.shape[1])

  # Pixelwerte als Knotenmerkmale in einen TensorFlow Tensor umwandeln
  node_features = tf.convert_to_tensor(image.reshape((-1, 3)).astype(np.float32))

  # Graphinformationen (Knotenmerkmale, Kanten, Kantengewichte) zurückgeben
  graph_info = (node_features, edges, edge_weights)

  return graph_info

In [ ]:
# Bilder in Graphen umwandeln
train_graphs = [image_to_graph(img) for img in x_train]

test_graphs = [image_to_graph(img) for img in x_test]

In [ ]:
# Den ersten Graphen im Trainingsdatensatz ausgeben (zur Überprüfung der Struktur)
train_graphs[0]

## GNN-Architektur

In [ ]:
def create_fcnn(hidden_units):
    # Eine Liste für die Schichten des Fully Connected Neural Network (FCNN) erstellen
    fcnn_layers = []

    # Schichten hinzufügen: Batch-Normalisierung und Dense-Schicht mit ReLU-Aktivierung
    for units in hidden_units:
        fcnn_layers.append(layers.BatchNormalization())
        fcnn_layers.append(layers.Dense(units, activation = tf.nn.relu))

    # Das FCNN als Keras Sequential Modell zurückgeben
    return keras.Sequential(fcnn_layers)

In [ ]:
class GraphConvLayer(layers.Layer):
    # Initialisierung der Graph Convolutional Layer
    def __init__(
        self,
        hidden_units,
        aggregation_type = "mean",
        combination_type = "concat",
        normalize = False,
        *args, **kwargs):
        super().__init__(*args, **kwargs)

        # Attribute für Aggregations-, Kombinations- und Normalisierungstyp festlegen
        self.aggregation_type = aggregation_type
        self.combination_type = combination_type
        self.normalize = normalize

        # FCNN für die Vorverarbeitung der Nachrichten erstellen
        self.fcnn_prepare = create_fcnn(hidden_units)
        # FCNN für die Aktualisierung der Knoten-Embeddings erstellen
        self.update_fn = create_fcnn(hidden_units)

    # Methode zur Vorverarbeitung der Knotenrepräsentationen
    def prepare(self, node_repesentations, weights = None):
        b, num_edges, embedding_dim = node_repesentations.shape
        # Nachrichten mit dem Vorverarbeitungs-FCNN berechnen
        messages = self.fcnn_prepare(node_repesentations)
        # Optional: Nachrichten mit Kantengewichten multiplizieren
        if weights is not None:
            messages = messages * tf.reshape(weights, (1, num_edges, 1))

        return messages

    # Methode zur Aggregation der Nachrichten von Nachbarknoten
    def aggregate(self, node_indices, neighbour_messages, node_repesentations):
        num_nodes = node_repesentations.shape[1]
        b, num_edges, embedding_dim = neighbour_messages.shape

        # Nachrichten für die Aggregation umformen
        neighbour_messages = tf.reshape(tf.transpose(neighbour_messages, (1, 0, 2)), (num_edges, -1))

        # Nachrichten basierend auf den Knotenindizes aggregieren (hier: Mittelwert)
        aggregated_message = tf.math.unsorted_segment_mean(neighbour_messages, node_indices, num_segments = num_nodes)
        # Aggregierte Nachrichten in die ursprüngliche Form zurückformen
        aggregated_message = tf.reshape(aggregated_message, (num_nodes, -1, embedding_dim))
        aggregated_message = tf.transpose(aggregated_message, (1, 0, 2))

        return aggregated_message

    # Methode zur Aktualisierung der Knotenrepräsentationen
    def update(self, node_repesentations, aggregated_messages):
        # Knotenrepräsentationen und aggregierte Nachrichten zusammenführen
        h = tf.concat([node_repesentations, aggregated_messages], axis = -1)
        # Knoten-Embeddings mit der Update-Funktion berechnen
        node_embeddings = self.update_fn(h)

        return node_embeddings

    # Call-Methode für die Ausführung der Graph Convolutional Layer
    def call(self, inputs):
        # Eingaben entpacken: Knotenrepräsentationen, Kanten, Kantengewichte
        node_repesentations, edges, edge_weights = inputs
        # Knoten- und Nachbarknotenindizes extrahieren
        node_indices, neighbour_indices = edges[0], edges[1]
        # Repräsentationen der Nachbarknoten sammeln
        neighbour_repesentations = tf.gather(node_repesentations, neighbour_indices, batch_dims=0, axis=1)

        # Nachrichten von Nachbarn vorbereiten
        neighbour_messages = self.prepare(neighbour_repesentations, edge_weights)
        # Nachrichten aggregieren
        aggregated_messages = self.aggregate(node_indices, neighbour_messages, node_repesentations)

        # Knotenrepräsentationen aktualisieren und zurückgeben
        return self.update(node_repesentations, aggregated_messages)

In [ ]:
class GNNGraphClassifier(tf.keras.Model):
    # Initialisierung des GNN Graph Classifiers
    def __init__(
        self,
        graph_info,
        hidden_units,
        aggregation_type = "sum",
        combination_type = "concat",
        normalize = True,
        *args, **kwargs):
        super().__init__(*args, **kwargs)

        # Graphinformationen extrahieren
        node_features, edges, edge_weights = graph_info
        # Graphinformationen als Attribute speichern
        self.node_features = node_features
        self.edges = edges
        self.edge_weights = edge_weights
        # Kantengewichte normalisieren
        self.edge_weights = self.edge_weights / tf.math.reduce_sum(self.edge_weights)

        # FCNN für die Vorverarbeitung der Knotenmerkmale erstellen
        self.preprocess = create_fcnn(hidden_units)
        # Erste Graph Convolutional Layer erstellen
        self.conv1 = GraphConvLayer(hidden_units, aggregation_type, combination_type, normalize)
        # Zweite Graph Convolutional Layer erstellen
        self.conv2 = GraphConvLayer(hidden_units, aggregation_type, combination_type, normalize)
        # FCNN für die Nachverarbeitung der Knoten-Embeddings erstellen
        self.postprocess = create_fcnn(hidden_units)
        # Dense-Schicht zur Berechnung der Sigmoid-Ausgabe (Klassifizierung) erstellen
        self.compute_sigmoid = layers.Dense(units = 1)

    # Call-Methode für die Ausführung des GNN Graph Classifiers
    def call(self, node_features):
        # Knotenmerkmale vorverarbeiten
        x = self.preprocess(node_features)
        # Erste Graph Convolution anwenden und Residual Connection hinzufügen
        x1 = self.conv1((x, self.edges, self.edge_weights))
        x = x1 + x
        # Zweite Graph Convolution anwenden und Residual Connection hinzufügen
        x2 = self.conv2((x, self.edges, self.edge_weights))
        x = x2 + x
        # Knoten-Embeddings nachverarbeiten
        x = self.postprocess(x)

        # Graph-Embedding durch Mittelwertbildung über die Knoten-Embeddings berechnen
        graph_embedding = tf.reduce_mean(x, axis = 1, keepdims = False)

        # Sigmoid-Ausgabe für die Klassifizierung berechnen und zurückgeben
        return self.compute_sigmoid(graph_embedding)

In [ ]:
# Instanz des GNNGraphClassifier Modells erstellen
# Hier wird der erste Graph aus den Trainingsdaten als Beispiel für die Graphstruktur verwendet
gnn_model = GNNGraphClassifier(
    graph_info = train_graphs[0],
    hidden_units = [32]) # Anzahl der Einheiten in den versteckten Schichten festlegen

## Modelltraining und -evaluation

In [ ]:
# Das GNN-Modell kompilieren
gnn_model.compile(optimizer = "adam", # Optimizer festlegen (hier: Adam)
              loss = "binary_crossentropy", # Verlustfunktion festlegen (hier: Binary Crossentropy für binäre Klassifizierung)
              metrics = ["accuracy"]) # Metriken für die Evaluation festlegen (hier: Genauigkeit)

In [ ]:
# Trainingsdaten in ein NumPy-Array umwandeln (nur die Knotenmerkmale)
x_train = np.array([tg[0] for tg in train_graphs], dtype = np.float32)

# Das GNN-Modell trainieren
gnn_model.fit(x_train, y_train, epochs = 5, batch_size = 32) # Anzahl der Epochen und Batch-Größe festlegen

In [ ]:
# Testdaten in ein NumPy-Array umwandeln (nur die Knotenmerkmale)
x_test = np.array([tg[0] for tg in test_graphs], dtype = np.float32),

# Das trainierte GNN-Modell auf den Testdaten evaluieren und die Ergebnisse ausgeben
eval_results = gnn_model.evaluate(x_test, y_test)
print("[test loss, test accuracy]:", eval_results)